# DropBlock: A regularization method for convolutional networks
### Paper Reproduction

In [7]:
from keras import backend as K
from keras.engine.topology import Layer
from scipy.stats import bernoulli
import copy

Using TensorFlow backend.


In [8]:
class DropBlock(Layer):
    """
    Regularization Technique for Convolutional Layers.
    
    Pseudocode:
    1: Input:output activations of a layer (A), block_size, γ, mode
    2: if mode == Inference then
    3: return A
    4: end if
    5: Randomly sample mask M: Mi,j ∼ Bernoulli(γ)
    6: For each zero position Mi,j , create a spatial square mask with the center being Mi,j , the width,
        height being block_size and set all the values of M in the square to be zero (see Figure 2).
    7: Apply the mask: A = A × M
    8: Normalize the features: A = A × count(M)/count_ones(M)
    
    # Arguments
        block_size: A Python integer. The size of the block to be dropped.
        gamma: float between 0 and 1. controls how many activation units to drop.
        seed: A Python integer to use as random seed.
    # References
        - [DropBlock: A regularization method for convolutional networks](
           https://arxiv.org/pdf/1810.12890v1.pdf)
    """
    def __init__(self, block_size, keep_prob, seed=None, **kwargs):
        super(DropBlock, self).__init__(**kwargs)
        self.block_size = block_size
        self.keep_prob = keep_prob
        self.seed = seed

    def call(self, x, training=None):
        
        # During inference, we do not Drop Blocks. (Similar to DropOut)
        if training == None:
            return x
        
        # Calculate Gamma
        feat_size = x.shape[0]
        gamma = (1-self.keep_prob / self.block_size**2) * (feat_size**2 / (feat_size-self.block_size+1)**2)
        
        padding = self.block_size//2
        
        # Randomly sample mask
        sample = bernoulli.rvs(size=(feat_size-padding, feat_size-padding),p=gamma)
    
        # For each 0, create spatial square mask of shape (block_size x block_size)
        mask = copy.copy(sample)
        for x in range(height):
            for y in range(width):
                if sample[x, y]==0:
                    mask[x-padding : x+padding+1, y-padding : y+padding+1] = 0
        
        # Pad the mask with ones
        mask = np.pad(mask, pad_width=1, mode='constant', constant_values=1)
        
        # Apply the mask
        x = x * mask
        
        # Normalize the features
        count = np.prod(a.shape)
        count_ones = np.count_nonzero(a == 1)
        x = x * count / count_ones
        
        return x
        
    def get_config(self):
        config = {'block_size': self.block_size,
                  'gamma': self.gamma,
                  'seed': self.seed}
        base_config = super(DropBlock, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
        
    def compute_output_shape(self, input_shape):
        return input_shape